In [1]:
import gym 
import random 
import numpy as np
import keras 
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam 

#ENV_NAME = "CartPole-v0"
#env=gym.make(ENV_NAME)

Using TensorFlow backend.


In [2]:
EPISODES = 250

In [3]:
class DQNAgent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=3000)
        self.gamma = 0.96   #discount_rate
        self.explorate = 1.0   #exploration rate
        self.explorate_min = 0.01
        self.explorate_decay = 0.993
        self.learning_rate = 0.001
        self.model = self._build_model()     
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24,input_dim = self.state_size,activation = 'relu'))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(24,activation='linear'))
        model.add(Dense(self.action_size,activation='softmax'))
        model.compile(loss='mse',
                     optimizer = Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        if np.random.rand() <= self.explorate:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  #return actions 
    
    def replay(self,batch_size):
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            target = reward
            if not done :
                target = (reward+self.gamma *np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action]=target
            self.model.fit(state,target_f,epochs = 1,verbose = 0)
        if self.explorate >self.explorate_min:
            self.explorate *= self.explorate_decay
    
    def load(self,name):
        self.model.load_weights(name)
        
    def save(self,name):
        self.model.save_weights(name)
                

In [4]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 36

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.explorate))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

episode: 0/250, score: 24, e: 1.0
episode: 1/250, score: 32, e: 0.86
episode: 2/250, score: 12, e: 0.79
episode: 3/250, score: 19, e: 0.69
episode: 4/250, score: 43, e: 0.51
episode: 5/250, score: 15, e: 0.46
episode: 6/250, score: 10, e: 0.43
episode: 7/250, score: 9, e: 0.4
episode: 8/250, score: 10, e: 0.38
episode: 9/250, score: 26, e: 0.31
episode: 10/250, score: 14, e: 0.28
episode: 11/250, score: 8, e: 0.27
episode: 12/250, score: 9, e: 0.25
episode: 13/250, score: 9, e: 0.24
episode: 14/250, score: 14, e: 0.21
episode: 15/250, score: 8, e: 0.2
episode: 16/250, score: 18, e: 0.18
episode: 17/250, score: 14, e: 0.16
episode: 18/250, score: 12, e: 0.15
episode: 19/250, score: 10, e: 0.14
episode: 20/250, score: 16, e: 0.12
episode: 21/250, score: 9, e: 0.12
episode: 22/250, score: 29, e: 0.095
episode: 23/250, score: 8, e: 0.09
episode: 24/250, score: 9, e: 0.084
episode: 25/250, score: 10, e: 0.079
episode: 26/250, score: 17, e: 0.07
episode: 27/250, score: 33, e: 0.055
episode: 

episode: 227/250, score: 14, e: 0.01
episode: 228/250, score: 14, e: 0.01
episode: 229/250, score: 7, e: 0.01
episode: 230/250, score: 31, e: 0.01
episode: 231/250, score: 29, e: 0.01
episode: 232/250, score: 7, e: 0.01
episode: 233/250, score: 9, e: 0.01
episode: 234/250, score: 8, e: 0.01
episode: 235/250, score: 10, e: 0.01
episode: 236/250, score: 23, e: 0.01
episode: 237/250, score: 8, e: 0.01
episode: 238/250, score: 45, e: 0.01
episode: 239/250, score: 8, e: 0.01
episode: 240/250, score: 8, e: 0.01
episode: 241/250, score: 26, e: 0.01
episode: 242/250, score: 14, e: 0.01
episode: 243/250, score: 20, e: 0.01
episode: 244/250, score: 33, e: 0.01
episode: 245/250, score: 8, e: 0.01
episode: 246/250, score: 28, e: 0.01
episode: 247/250, score: 8, e: 0.01
episode: 248/250, score: 22, e: 0.01
episode: 249/250, score: 9, e: 0.01


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\events.py", line 145, in _run
    self._callback(*self._args)
  File "C:\Users\dell\Anaconda3\envs\tensorflow\lib\asyncio\selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。
Exception in callback BaseSelectorEventLoop._r

In [1]:
import random

class Q_learner:
    def __init__(self, R):
		#reward
		self.R = R
		#Q function
		self.Q = [[0]*6 for _ in range(6)]
		#future discount
		self.gamma = 0.8
		#final state
		self.goal = 5

		# exploration rate
		self.epsilon = 1.0 
		self.epsilon_min = 0.01
		self.epsilon_decay = 0.999

		
	#displays the Q matrix		
	def display_Q(self):
		for i in self.Q:
			for j in i:
				print(j, end='\t')
			print()	
		print()


	#gets all the possible actions of a state
	def get_actions(self, state):
		poss_actions = []
		for i, reward in enumerate(self.R[state]):
			if (reward != -1):
				poss_actions.append(i)

		return poss_actions

	#selects the optimal action from the Q matrix
	def optimal_action(self, state, poss_actions):
		max_a = 0
		actions = []
		for i in poss_actions:
			temp = self.Q[state][i]
			if temp == max_a:
				actions.append(i)
			elif temp > max_a:
				actions = [i]
				max_a = temp
		return random.choice(actions)


	#with probabity epsilon, does a random action
	#else selects optimal action
	def act(self, state, poss_actions):
		if random.random() <= self.epsilon:
		    return random.choice(poss_actions)
		return self.optimal_action(state, poss_actions)


	#converts Q matrix to percentage
	#divides each value in Q with max Q value 
	def to_percent(self):
		max_Q = 0
		for i in self.Q:
			temp = max(i)
			if (temp > max_Q):
				max_Q = temp
		for i, _ in enumerate(self.Q):
			self.Q[i][:] = [round(100 * x / max_Q) for x in self.Q[i]]


	#finds the optimal Q matrix
	def train(self, iterations=2000):
		#initial state 
		state = random.randint(0, 5)

		for _ in range(iterations):
			#all the possible actions the state has
			poss_actions = self.get_actions(state)

			#selecting an action
			action = self.act(state, poss_actions)

			#perform the action, i.e. going to the new state
			new_state = action

			#updating the Q matrix
			self.Q[state][action] = self.R[state][action] + self.gamma * max(self.Q[new_state])

			#decaying epsilon
			if self.epsilon > self.epsilon_min:
				self.epsilon *= self.epsilon_decay

			#if goal has been reached, we stop
			if new_state == self.goal:
				#select a new random initial state
				state = random.randint(0, 5)
				continue

			#else we change state and continue
			state = new_state

		#convert each value in Q matrix to percent
		self.to_percent()


	#finds optimal path from given state to goal
	def find_path(self, initial_state):
		state = initial_state
		print(state, end=' -> ')
		while True:
			poss_actions = self.get_actions(state)
			action = self.optimal_action(state, poss_actions)
			state = action
			if state == self.goal:
				print(state)
				break

			print(state, end=' -> ')

def main():
	#reward
	R = [
			[-1, -1, -1, -1, 0, -1],
			[-1, -1, -1, 0, -1 , 100],
			[-1, -1, -1, 0, -1, -1],
			[-1, 0, 0, -1, 0, -1],
			[0, -1, -1, 0, -1, 100],
			[-1, 0, -1, -1, 0, 100]
	]

	env = Q_learner(R)
	env.train()

	print('\nFinal Q matrix:')
	env.display_Q()

	env.find_path(2)

if __name__ == '__main__':
	main()



Final Q matrix:
0	0	0	0	80	0	
0	0	0	64	0	100	
0	0	0	64	0	0	
0	80	51	0	80	0	
64	0	0	64	0	100	
0	80	0	0	80	100	

2 -> 3 -> 4 -> 5
